<a href="https://colab.research.google.com/github/arqMedes/aulas/blob/master/Grafo_Circuitos_Eletricos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Teoria de Circuitos Elétricos

![](https://github.com/arqMedes/images/blob/main/grafo_circuito_eletrico.jpg?raw=true)


🔹 Etapa 1: Representação de um ramo

Cada ramo precisa guardar informações como:

**tipo:** (Resistor, Capacitor, Indutor, Fonte de Tensão, Fonte de Corrente, etc.)

**valor:** (resistência, capacitância, etc.)

**Vs:** (fonte de tensão equivalente do ramo)

**Is:** (fonte de corrente equivalente do ramo)

**corrente do ramo (I):**

**tensão do ramo (V):**


🔹 Etapa 2: Montagem da lista de ramos

🔹 Etapa 3: Determinar a lista de Nós e a Matriz de Incidência A

Se temos N nós e M ramos, a matriz de incidência A é uma matriz N × M onde:

+1 → ramo sai do nó

-1 → ramo entra no nó

0 → ramo não conectado


🔹 Etapa 4: Montar Vs, Is, G

🔹 Etapa 5: Cálculos (Método dos Nós Modificado)

$$G_b=A⋅G⋅A.T \\
𝐼_𝑛 = 𝐴⋅(𝐺⋅𝑉_𝑠−𝐼_𝑠) \\
E=G_b^{−1}.In \\
V=A.T⋅E  \\
I=I_s+G⋅(V−V_s)
$$

Implementação:


🔹 Exemplo de circuito

Fonte de tensão de 10 V (nó 1 para nó 0, que é a referência).

Dois resistores em série ligados a essa fonte:

R1 = 10 Ω (entre nó 1 e nó 2)

R2 = 5 Ω (entre nó 2 e nó 0)

Isso forma um divisor de tensão bem clássico.


🔹 Resultado esperado

Como é um divisor de tensão de 10 V com resistores 10 Ω e 5 Ω:



In [11]:
import numpy as np

class grafo:
  def __init__(self, ramosCircuito):
    self.ramos = ramosCircuito
    self.nos = set([])
    self._nosCircuito()
    self._Calculos()

  def _nosCircuito(self):
    for k, r in enumerate(self.ramos):
      no1 = r.NP
      no2 = r.NC

      self.nos.add(no1)
      self.nos.add(no2)

  def listaNos(self):
    return list(self.nos)

  def _Calculos(self):
    A = np.zeros((len(self.nos)-1, len(self.ramos)))
    G = np.zeros((len(self.ramos), len(self.ramos)))
    Vs = np.zeros((len(self.ramos), 1))
    Is = np.zeros((len(self.ramos), 1))

    for k, r in enumerate(self.ramos):
      no1 = r.NP
      no2 = r.NC

      if (no1 != 0):
        A[no1-1, k ] = 1.
      if (no2 != 0):
        A[no2-1, k ] = -1.

      G[k, k] = r.condutancia()
      Vs[k, 0] = r.Vs
      Is[k, 0] = r.Is

    self._matricialmente(A, G, Vs, Is)

  def _matricialmente(self, A, G, Vs, Is):

    # Matrizes
    A = np.matrix(A, dtype=np.float64)
    G = np.matrix(G, dtype=np.float64)
    Vs = np.matrix(Vs, dtype=np.float64)
    Is = np.matrix(Is, dtype=np.float64)

    '''

    print(100*"*")
    print("Entrada ")
    print("\nA \n", A)
    print("\nG \n", G)
    print("\nVs \n", Vs)
    print("\nIs \n", Is)

    '''

    Gb = A*G*A.T
    In = A*(G*Vs - Is)
    E = (Gb.I)*In
    V = A.T*E
    I = Is + G*(V - Vs)

    '''

    print(100*"*")
    print("\nSaída ")
    print("\nCondutâncias de barras \n", Gb)
    print("\nCorrente Independente Resultante no Nó \n", In)
    print("\nTensão de Nó \n", E)
    print("\nTensão de ramo \n", V)
    print("\nCorrente de ramo \n", I)
    '''
    for k, r in enumerate(self.ramos):
      r.I = I[k, 0]
      r.V = V[k, 0]

  def imprime(self):
    for k, r in enumerate(self.ramos):
      print("Ramo: ", k+1)
      print("Tipo: ", r.tipo)
      print("NP: ", r.NP)
      print("NC: ", r.NC)
      print("Valor: ", r.valor)
      print("Vs: ", r.Vs)
      print("Is: ", r.Is)
      print("I: ", r.I)
      print("V: ", r.V)
      print(100*"*")



class Ramo:
    def __init__(self, tipo, NP, NC, valor, Vs=0.0, Is=0.0):
        self.tipo = tipo
        self.NP=NP
        self.NC=NC
        self.valor = valor
        self.Vs = Vs   # fonte de tensão
        self.Is = Is   # fonte de corrente
        self.I = 0.0   # corrente no ramo
        self.V = 0.0   # tensão no ramo

    def condutancia(self):

      if self.tipo.lower() == "resistor" and self.valor != 0:
        return 1.0 / self.valor

      if self.tipo.lower() == "fonte de tensão" and self.valor == 0:
        return 1.0e9

      if self.tipo.lower() == "fonte de corrente" and self.valor == 0:
        return 1.0-9

      if self.tipo.lower() == "genérico":
        return self.valor




In [12]:
circuito = [
    Ramo("Resistor",1, 2, 10),          # ramo 0
    Ramo("Resistor",2, 0, 5),           # ramo 1
    Ramo("Fonte de Tensão",1,0, 0, 10) # ramo 2
]
G = grafo(circuito)
G.listaNos()
G.imprime()

Ramo:  1
Tipo:  Resistor
NP:  1
NC:  2
Valor:  10
Vs:  0.0
Is:  0.0
I:  0.6666666666222221
V:  6.666666666222221
****************************************************************************************************
Ramo:  2
Tipo:  Resistor
NP:  2
NC:  0
Valor:  5
Vs:  0.0
Is:  0.0
I:  0.6666666666222222
V:  3.333333333111111
****************************************************************************************************
Ramo:  3
Tipo:  Fonte de Tensão
NP:  1
NC:  0
Valor:  0
Vs:  10
Is:  0.0
I:  -0.6666684981837534
V:  9.999999999333332
****************************************************************************************************


![](https://camo.githubusercontent.com/aea48362d41f072bff5d0c3f8a14ef7083b780d8a3237557052fddfe7ff1a4fb/68747470733a2f2f6769746875622e636f6d2f61727150792f4341442f626c6f622f6d61696e2f436972637569746f5f656c65747269636f2f6669672f616e616c6973654e6f64616c2e706e673f7261773d74727565)

![](https://github.com/arqMedes/images/blob/main/Ramo_circuitoX.jpg?raw=true)

In [13]:
circuito = [
    Ramo("Genérico", 3, 1, 7/10, Is= 1),
    Ramo("Genérico", 1, 2, 1/5, Is= -2),
    Ramo("Genérico", 2, 3, 1/10),
    Ramo("Genérico", 0, 2, 1/4, Is= 3),
    Ramo("Genérico", 0, 3, 1/2)
]
G = grafo(circuito)
G.listaNos()
G.imprime()

Ramo:  1
Tipo:  Genérico
NP:  3
NC:  1
Valor:  0.7
Vs:  0.0
Is:  1
I:  -1.5789473684210518
V:  -3.6842105263157885
****************************************************************************************************
Ramo:  2
Tipo:  Genérico
NP:  1
NC:  2
Valor:  0.2
Vs:  0.0
Is:  -2
I:  -1.5789473684210529
V:  2.1052631578947354
****************************************************************************************************
Ramo:  3
Tipo:  Genérico
NP:  2
NC:  3
Valor:  0.1
Vs:  0.0
Is:  0.0
I:  0.1578947368421053
V:  1.578947368421053
****************************************************************************************************
Ramo:  4
Tipo:  Genérico
NP:  0
NC:  2
Valor:  0.25
Vs:  0.0
Is:  3
I:  1.7368421052631575
V:  -5.05263157894737
****************************************************************************************************
Ramo:  5
Tipo:  Genérico
NP:  0
NC:  3
Valor:  0.5
Vs:  0.0
Is:  0.0
I:  -1.7368421052631584
V:  -3.473684210526317
***********************